# README.md

# Assign ADT window for the 270 pts from previously calculated results approach #2 and manual reviewed cohort and mark the sources

## Table of Contents
* [Read data, clean up and get stats](#first-bullet)
* [Assign Source and combine tables](#second-bullet)

In [112]:
import numpy as np
import pandas as pd
import sys, getopt
import csv
from datetime import datetime
import itertools

import matplotlib.pyplot as plt
from matplotlib import pyplot
from pylab import *

%matplotlib inline  
import seaborn as sns

Note: All the pts who made it to the final cohort met the following criteria:

Having an initial ADT window assignable with start and end dates
Having a baseline T before or on the day of ADT start date
Having T test results after or on the  ADT end date that a recovery/nonrecovery status can be assigned
ADT_to_T recovery time > 0 days
ADT duration >0 days


# Read data, clean up and get stats <a class="anchor" id="first-bullet"></a>

In [113]:
#new 107pt list
df1=pd.read_excel("./data/270pt_list.xlsx", header=0)

#manual reviewed cohort 
df2=pd.read_excel("./data/pca_manual_reviewed_cohort_185pts_ADT_windows.xlsx", header=0)

#approach #2 results
df3=pd.read_csv("./data/sean_big_cohort_approach2_561pts_with_adt_window_and_baseline_T_Feb24_2021.csv", header=0)

#103/185 pts that have T baselines
df4=pd.read_excel("./data/103pts_combined_185pts_cohort_with_baselineT.xlsx", header=0)

#84/103 pts that have T baselines
df5=pd.read_excel("./data/84pts_manual_reviewed_final_cohort.xlsx", header=0)

#252 pts in the approach #2 final cohort patient list 
df6=pd.read_excel("./data/approach2_252pts_final_cohort.xlsx", header=0)

df1.fillna('', inplace=True)
df1.replace(r'', np.NaN)

df2.fillna('', inplace=True)
df2.replace(r'', np.NaN)
df2[:3]

df3.fillna('', inplace=True)
df3.replace(r'', np.NaN)
df3[:3]

df5.fillna('', inplace=True)
df5.replace(r'', np.NaN)
df5[:3]
print(len(df5))

df6.fillna('', inplace=True)
df6.replace(r'', np.NaN)
df6[:3]
print(len(df6))

84
252


In [114]:
df2[:3]

,person_id,ADT initail use START,ADT initial use END,ADT duration,Last note date
0,12,2013-05-06,2013-05-06,0,2020-12-18
1,18,2013-11-12,2014-03-13,121,2020-10-29
2,25,2013-08-01,2014-02-18,201,2020-01-22


In [115]:
#first assgin window according to the manual reviewed cohort
#then for the empty lines, assgin them according to the approach #2 
result_pt_list=df1["person_id"].to_list()
manual_pt_list=df2["person_id"].to_list()
approach2_list=df3["person_id"].to_list()
manual_final=df5["person_id"].to_list()
app2_final=df6["person_id"].to_list()

#overlapping btw 270 pts list and  manual reviewed list with T baseline
shared_btw_manual_n_app2=[x for x in app2_final if x in manual_final]
print(len(shared_btw_manual_n_app2))
unique_from_manual=[x for x in manual_final if x not in app2_final]
unique_from_app2=[x for x in app2_final if x not in manual_final]

print(len(unique_from_manual))
print(len(unique_from_app2))

66
18
186


In [116]:
252+84-66

270

In [117]:
len(manual_pt_list)

185

In [118]:
df3

,person_id,inital_ADT_start_date,inital_ADT_end_date,Inital_ADT_time_window_days,Tdate_justbeforeADT,Tvalue_justbeforeADT,baseline avail Status,N_T_dates,N_T_dates_beforeADT
0,2,2009-02-26,2009-09-15,201.0,2009-02-09,274.18,Y,82,20
1,13,2010-06-17,2015-03-17,1734.0,2009-09-18,23.72,Y,23,2
2,25,2013-11-01,2014-02-18,109.0,2013-10-15,31.54,Y,12,1
3,38,2015-11-18,2016-07-18,243.0,2015-11-18,288.37,Y,20,5
4,53,2013-02-25,2016-10-03,1316.0,2012-07-30,0.96,Y,26,1
...,...,...,...,...,...,...,...,...,...
502,197945,2020-06-02,2020-09-09,99.0,2020-06-02,267.00,Y,1,1
503,198103,2020-01-16,2020-09-21,249.0,2020-01-16,610.00,Y,2,1
504,201343,2020-08-07,2020-09-01,25.0,2020-07-09,362.00,Y,3,1
505,203736,2020-07-07,2020-10-14,99.0,2020-07-06,823.00,Y,1,1


In [119]:
df3.columns

for column in ['Tdate_justbeforeADT','Tvalue_justbeforeADT', 'baseline avail Status', 'N_T_dates','N_T_dates_beforeADT']:
    df3 = df3.drop(column, 1)

df3.columns=['person_id','ADT_start','ADT_end','ADT_duration']
df3[:3]

,person_id,ADT_start,ADT_end,ADT_duration
0,2,2009-02-26,2009-09-15,201.0
1,13,2010-06-17,2015-03-17,1734.0
2,25,2013-11-01,2014-02-18,109.0


In [120]:
shape(df3)

(507, 4)

In [121]:
index_list=df2.columns
index_list

df2.columns=['person_id','ADT_start','ADT_end','ADT_duration', 'Last_note_date']
df2 = df2.drop('Last_note_date', 1)

In [122]:
df2[:30]
#print(len(df2))
y=38
df2_pt=df2[(df2["person_id"] == y)]
start=df2_pt.ADT_start.to_list()[0]
start

Timestamp('2015-11-18 00:00:00')

In [123]:
df2.ADT_start.notnull()

0      True
1      True
2      True
3      True
4      True
       ... 
180    True
181    True
182    True
183    True
184    True
Name: ADT_start, Length: 185, dtype: bool

# Assign Source and combine tables<a class="anchor" id="first-bullet"></a>

In [124]:
part1=df2[(df2["person_id"].isin(shared_btw_manual_n_app2))] 
part1['Source']='shared_btw_manual_n_app2'
print(len(part1))
part1[:3]

66


/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,person_id,ADT_start,ADT_end,ADT_duration,Source
4,38,2015-11-18,2016-07-18,243,shared_btw_manual_n_app2
7,119,2013-07-02,2013-10-02,92,shared_btw_manual_n_app2
10,780,2010-08-01,2010-09-30,60,shared_btw_manual_n_app2


In [125]:
part2=df2[(df2["person_id"].isin(unique_from_manual))] 
part2['Source']='uniquely_from_manual'
print(len(part2))
part2[:3]

18


/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,person_id,ADT_start,ADT_end,ADT_duration,Source
5,81,2014-04-15,2016-06-09,786,uniquely_from_manual
19,2660,2018-07-03,2018-10-09,98,uniquely_from_manual
30,7236,2016-08-19,2017-01-04,138,uniquely_from_manual


In [126]:
part3=df3[(df3["person_id"].isin(unique_from_app2))] 
part3['Source']='uniquely_from_app2'
print(len(part3))
part3[:3]

186


/Users/yuqincai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,person_id,ADT_start,ADT_end,ADT_duration,Source
0,2,2009-02-26,2009-09-15,201.0,uniquely_from_app2
1,13,2010-06-17,2015-03-17,1734.0,uniquely_from_app2
2,25,2013-11-01,2014-02-18,109.0,uniquely_from_app2


In [127]:
Final_270pts=pd.concat([part1,part2,part3], ignore_index=True)
Final_270pts

,person_id,ADT_start,ADT_end,ADT_duration,Source
0,38,2015-11-18 00:00:00,2016-07-18 00:00:00,243.0,shared_btw_manual_n_app2
1,119,2013-07-02 00:00:00,2013-10-02 00:00:00,92.0,shared_btw_manual_n_app2
2,780,2010-08-01 00:00:00,2010-09-30 00:00:00,60.0,shared_btw_manual_n_app2
3,870,2017-06-14 00:00:00,2017-10-13 00:00:00,121.0,shared_btw_manual_n_app2
4,1257,2014-10-02 00:00:00,2015-07-08 00:00:00,279.0,shared_btw_manual_n_app2
...,...,...,...,...,...
265,185086,2019-09-10,2020-02-07,150.0,uniquely_from_app2
266,187363,2019-02-01,2019-06-19,138.0,uniquely_from_app2
267,187900,2019-12-30,2020-02-03,35.0,uniquely_from_app2
268,189405,2019-05-01,2019-07-29,89.0,uniquely_from_app2


In [128]:
Final_270pts.to_csv("./data/combined_270pts_from_manual_reviewed_n_approach2_with_ADT.csv", encoding='utf-8', index=False)